In [5]:
import json
import csv
import pandas as pd
import numpy as np
import nltk
import string
import re
import gensim

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)



In [2]:
#Brining in arxiv metadata as CSV
#df = pd.read_csv("/Users/aster/Desktop/Fall 2020/ML/final_project/data/arxiv_metadata.csv")

In [3]:
#df.head()

In [30]:
#Setting up Paths for rest of Notebook
import os
import os.path

#'uncleaned' subset of from arxiv metadata snapshot. ML(cs.lg) and or AI(cs.ai) category tagged papers with "fair" in the title
#/Users/aster/Desktop/fall_2020/ML/final_project/pratt_ml_final_project/data/arxiv_corpus_2020_11_23.csv
snapshot = os.path.join("/Users/aster/Desktop/fall_2020/ML/final_project/data/arxiv_corpus_2020_11_23.csv")

#data after cleaning, lemmatization, bigrams, tokenization
dataset = os.path.join("/Users/aster/Desktop/fall_2020/ML/final_project/data/arxiv_corpus_2020_12_8.csv")

### Data Subset Selection

Now that we have the arxiv metadata snapshot as a csv we can make 2 corpus, one with abstract data and one with title; which we'll preprocess and fit our model. For this project we're particularly interested in how machine learning practitioners are thinking and talking about "fairness". We're trying to see if we can identify types of approaches to fairness in the form of subjects. 

Our data subset will include papers: 
- With "fair" in the title
- With machine learning (cs.LG) and artificial intelligence (cs.AI) in the categories column

In this part of the project required loading the arxiv metadata snaphot which was rather large. I've commented it out here. 

In [5]:
#Papers that have fair in title
#df=df[df['title'].str.lower().str.contains("fair")]


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
1841,704.184,Inanc Inan,"Feyza Keceli, Inanc Inan, Ender Ayanoglu",Fairness Provision in the IEEE 802.11e Infrastructure Basic Service Set,NaN,NaN,NaN,NaN,cs.OH,NaN,"Most of the deployed IEEE 802.11e Wireless Local Area Networks (WLANs) use\ninfrastructure Basic Service Set (BSS) in which an Access Point (AP) serves as\na gateway between wired and wireless domains. We present the unfairness problem\nbetween the uplink and the downlink flows of any Access Category (AC) in the\n802.11e Enhanced Distributed Channel Access (EDCA) when the default settings of\nthe EDCA parameters are used. We propose a simple analytical model to calculate\nthe EDCA parameter settings that achieve weighted fair resource allocation for\nall uplink and downlink flows. We also propose a simple model-assisted\nmeasurement-based dynamic EDCA parameter adaptation algorithm. Moreover, our\ndynamic solution addresses the differences in the transport layer and the\nMedium Access Control (MAC) layer interactions of User Datagram Protocol (UDP)\nand Transmission Control Protocol (TCP). We show that proposed Contention\nWindow (CW) and Transmit Opportunity (TXOP) limit adaptation at the AP provides\nfair UDP and TCP access between uplink and downlink flows of the same AC while\npreserving prioritization among ACs.\n","[{'version': 'v1', 'created': 'Fri, 13 Apr 2007 23:11:27 GMT'}, {'version': 'v2', 'created': 'Thu, 13 Sep 2007 15:08:17 GMT'}]",2007-09-13,"[['Keceli', 'Feyza', ''], ['Inan', 'Inanc', ''], ['Ayanoglu', 'Ender', '']]"
4441,705.044,Marco Genovese,"G. Brida, M. Genovese, F.Piacentini",Experimental local realism tests without fair sampling assumption,NaN,"EPJD, Vol. 4, Issue 3 (2007)",10.1140/epjd/e2007-00194-5,NaN,quant-ph,NaN,"Following the theoretical suggestion of Ref. [1,2], we present experimental\nresults addressed to test restricted families of local realistic models, but\nwithout relying on the fair sampling assumption.\n","[{'version': 'v1', 'created': 'Thu, 3 May 2007 12:17:51 GMT'}, {'version': 'v2', 'created': 'Thu, 10 May 2007 08:39:19 GMT'}]",2009-11-13,"[['Brida', 'G.', ''], ['Genovese', 'M.', ''], ['Piacentini', 'F.', '']]"
5479,705.148,Guillaume Adenier,Guillaume Adenier,"Quantum entanglement, fair sampling, and reality: Is the moon there when\n nobody looks?","13 pages, 8 Figures, 1 Table","Am. J. Phys. Vol 76, 2, 147-152 (2008)",10.1119/1.2825326,NaN,quant-ph,NaN,"In 1981, David Mermin described a cleverly simplified version of Bell's\ntheorem. It pointed out in a straightforward way that interpreting entanglement\nfrom a local realist point of view can be problematic. I propose here an\nextended version of Mermin's device that can actually be given a simple local\nrealist interpretation through a sample selection bias, and I argue that we\nstill have no scientific reason to believe that the moon could possibly not be\nthere when nobody looks.\n","[{'version': 'v1', 'created': 'Thu, 10 May 2007 13:53:04 GMT'}, {'version': 'v2', 'created': 'Fri, 29 Jun 2007 22:06:00 GMT'}, {'version': 'v3', 'created': 'Fri, 10 Aug 2007 15:59:02 GMT'}]",2008-03-06,"[['Adenier', 'Guillaume', '']]"
10777,706.209,Qingfeng Li,"Qingfeng Li, Marcus Bleicher, and Horst Stoecker",Pion freeze-out as seen through HBT correlations in heavy ion collisions\n from FAIR/AGS to RHIC energies,"11 pages, 2 figs","J.Phys.G34:2037-2044,2007",10.1088/0954-3899/34/9/012,NaN,nucl-th,NaN,"We perform a systematic analysis of several HBT parameters in heavy ion\ncollisions from $E_{\rm beam}=2$ AGeV to $\sqrt {s_{\rm NN}}=200$ GeV within\nthe UrQMD transport approach and compare the results to experimental data where\navailable. We find that the 'lifetime' of the emission source as calculated\nfrom $\tau \sim \sqrt{R_O^{2}-R_S^{2}}$, is larger than the experimentally\nobserved values at all investigated energies. The calculated volume of the pion\nsou

In [6]:
#Papers including fair in the title that are in our categories of interest
df = df[df["categories"].str.lower().str.contains("cs.ai", "cs.lg")]
df.shape

(218, 14)

In [12]:
#Writing subset to a csv file in the data folder
df = pd.read_csv(snapshot, encoding = 'utf8')
df.head()

,Unnamed: 0,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,342381,1205.3380,Yefim Bakman,Yefim Bakman,Unfair items detection in educational measurement,"14 pages, 5 figures",NaN,NaN,NaN,cs.AI physics.ed-ph,http://arxiv.org/licenses/nonexclusive-distrib/1.0/,"Measurement professionals cannot come to an agreement on the definition of\nthe term 'item fairness'. In this paper a continuous measure of item unfairness\nis proposed. The more the unfairness measure deviates from zero, the less fair\nthe item is. If the measure exceeds the cutoff value, the item is identified as\ndefinitely unfair. The new approach can identify unfair items that would not be\nidentified with conventional procedures. The results are in accord with\nexperts' judgments on the item qualities. Since no assumptions about scores\ndistributions and/or correlations are assumed, the method is applicable to any\neducational test. Its performance is illustrated through application to scores\nof a real test.\n","[{'version': 'v1', 'created': 'Sat, 11 Feb 2012 16:12:05 GMT'}]",2020-10-06,"[['Bakman', 'Yefim', '']]"
1,414517,1303.2860,"Moritz M\""uhlenthaler","Moritz M\""uhlenthaler, Rolf Wanka",Fairness in Academic Course Timetabling,"appeared in PATAT 2012, pp. 114-130",NaN,10.1007/s10479-014-1553-2,NaN,cs.AI cs.DS,http://arxiv.org/licenses/nonexclusive-distrib/1.0/,"We consider the problem of creating fair course timetables in the setting of\na university. Our motivation is to improve the overall satisfaction of\nindividuals concerned (students, teachers, etc.) by providing a fair timetable\nto them. The central idea is that undesirable arrangements in the course\ntimetable, i.e., violations of soft constraints, should be distributed in a\nfair way among the individuals. We propose two formulations for the fair course\ntimetabling problem that are based on max-min fairness and Jain's fairness\nindex, respectively. Furthermore, we present and experimentally evaluate an\noptimization algorithm based on simulated annealing for solving max-min fair\ncourse timetabling problems. The new contribution is concerned with measuring\nthe energy difference between two timetables, i.e., how much worse a timetable\nis compared to another timetable with respect to max-min fairness. We introduce\nthree different energy difference measures and evaluate their impact on the\noverall algorithm performance. The second proposed problem formulation focuses\non the tradeoff between fairness and the total amount of soft constraint\nviolations. Our experimental evaluation shows that the known best solutions to\nthe ITC2007 curriculum-based course timetabling instances are quite fair with\nrespect to Jain's fairness index. However, the experiments also show that the\nfairness can be improved further for only a rather small increase in the total\namount of soft constraint violations.\n","[{'version': 'v1', 'created': 'Tue, 12 Mar 2013 12:46:54 GMT'}]",2015-05-01,"[['Mühlenthaler', 'Moritz', ''], ['Wanka', 'Rolf', '']]"
2,439783,1306.4999,Lizi Zhang,Lizi Zhang,Safeguarding E-Commerce against Advisor Cheating Behaviors: Towards More\n Robust Trust Models for Handling Unfair Ratings,NaN,NaN,NaN,NaN,cs.SI cs.AI,http://arxiv.org/licenses/nonexclusive-distrib/1.0/,"In electronic marketplaces, after each transaction buyers will rate the\nproducts provided by the sellers. To decide the most trustworthy sellers to\ntransact with, buyers rely on trust models to leverage these ratings to\nevaluate the reputation of sellers. Although the high effectiveness of\ndifferent trust models for handling unfair ratings have been claimed by their\ndesigners, recently it is argued that these models are vulnerable to more\nintelligent attacks, and there is an urgent demand that the robustness of the\nexisting trust models has to be evaluated in a more comprehensive way. In this\nwork, we classify the existing trust models into two broad

In [14]:
#using a lambda function to apply str(ing).lower(case) to the abstract and title column. 
#Also making new columns for the "cleaned" abstract and title data. This way we hold unto our original data.
#I'm using a lambda function because I need to apply this to multiple columns
df[["abstract_clean", "title_clean"]] = df[["abstract", "title"]].apply(lambda x: x.str.lower())

## Data Cleaning: 

Now that we have our subset data in one place we can move unto cleaning. 

- [x] I'd like to get rid of puntuations.  
- [x] I'd like to get rid of "/n"
- [x] I'd like to get rid of numbers


In [15]:
import re
import string

#creating a variable to hold all the string patterns we want to operate on with regex
# "|" pipe is an "or" joing the pattern for:
#punctuation "[^\w\s]"
#new line marker "\n"
#and numerals "\d+"
#then apply .str.replace via lambda function
pattern = '|'.join(["[^\w\s]", "\n", "\d+", "[‘’“”…]"])

df[['title_clean','abstract_clean']] = df[['title_clean','abstract_clean']].apply(lambda x: x.str.replace(pattern, ' '))
#df.head()


In [16]:
#Leading and Trailing White Space
df[['title_clean','abstract_clean']] = df[['title_clean','abstract_clean']].apply(lambda x: x.str.strip())
#df.head()

## Further Cleaning - More Normalization and Tokenization

Now that we have our corpus clean we can had into preprocessing for our model. 
Our Preprocessing Tasks are: 

- [x] Stop words
- [x] Stemming/Lemmatization of words
- [x] Tokenizing words - Creating Document Term Matrix (otherwise known as bag of words)
- [] Bigrams

In [17]:
### Stemming/Lemmatization 
#nltk.download('wordnet')
import nltk
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from gensim.models.phrases import Phraser


t = nltk.tokenize.WhitespaceTokenizer()
lem = nltk.stem.WordNetLemmatizer()

#this function will tokenize AND THEN lemmatize values in a column
def lem_text(text):
    return [lem.lemmatize(w, 'v') for w in t.tokenize(text)]

#df = pd.DataFrame([‘this was cheesy blessing’, 'she likes these books ', ‘wow this is great amazing’], columns=[‘text’])
#print(df)

df["title_clean"] = df["title_clean"].apply(lem_text)
df["abstract_clean"] = df["abstract_clean"].apply(lem_text)

#Now we have tokenized and lemmed columns of abstract adn title data, 

In [148]:
#df.head()

In [18]:
#Getting rid of Stop Words
from nltk.corpus import stopwords

stop=stopwords.words("english")

df["abstract_clean"]= df['abstract_clean'].apply(lambda x: [item for item in x if item not in stop])
df['title_clean'] = df['title_clean'].apply(lambda x: [item for item in x if item not in stop])
# Strangely I can't group these funtion together or it doesn't seem to work on both columns i'm tryin to select. 

In [20]:
#df.head()

### N-grams: Bigrams, Trigrams, Probability

In computational lingusitics/natural language processing an "n-gram" describes a sequence of n-items in a collection of text. 

So in the sentence:

"Machine Learning should reckon with fairness, justice, and equity."

"Machine" - 1-gram (unigram)

"Machine, Learning" - 2-gram (bigram)

"Machine, Learning, should" - 3-gram (trigram)

Machine, Learning, should, reckon" - 4-(quadgram)

etc. 

Currently the text we're looking at is tokenized as unigrams. So now we're going to try using bigrams and trigrams and see what sort of new results we can get. 


In [21]:
###Setting Bigrams
import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser

def bigrams(text):
    bigram = Phrases(text, min_count=1)
    bigram_mod = Phraser(bigram)
    return [bigram_mod[[x]] for x in text]

df['title_clean']=df['title_clean'].apply(lambda x: (x))
df['title_clean']=df[['title_clean']].apply(bigrams)


In [23]:
#df.head()

In [24]:
#Setting bigrams for the abstract data. We'll primarily be looking at abstracts for our topics.
df['abstract_clean']=df['abstract_clean'].apply(lambda x: (x))
df['abstract_clean']=df[['abstract_clean']].apply(bigrams)


In [26]:
#df.head()

In [27]:
#df['abstract_clean'][1]

In [28]:
list(df['abstract_clean'][1])

[['consider_problem',
  'create_fair',
  'course',
  'timetables',
  'set',
  'university',
  'motivation',
  'improve',
  'overall',
  'satisfaction',
  'individuals',
  'concern',
  'students',
  'teachers',
  'etc',
  'provide',
  'fair',
  'timetable',
  'central',
  'idea',
  'undesirable',
  'arrangements',
  'course',
  'timetable',
  'e',
  'violations',
  'soft',
  'constraints',
  'distribute',
  'fair',
  'way',
  'among',
  'individuals',
  'propose_two',
  'formulations',
  'fair_course',
  'timetabling',
  'problem',
  'base',
  'max_min',
  'fairness',
  'jain_fairness',
  'index',
  'respectively',
  'furthermore',
  'present',
  'experimentally_evaluate',
  'optimization',
  'algorithm',
  'base',
  'simulate',
  'anneal',
  'solve',
  'max_min',
  'fair_course',
  'timetabling',
  'problems',
  'new',
  'contribution',
  'concern',
  'measure',
  'energy_difference',
  'two',
  'timetables',
  'e',
  'much',
  'worse',
  'timetable',
  'compare',
  'another',
  'timet

In [149]:
df["title_clean"] = df["title_clean"].apply(lambda x: list(x)[0])

In [151]:
df["abstract_clean"] = df["abstract_clean"].apply(lambda x: list(x)[0])

In [154]:
#Checking the 2nd paper's title to make sure it's a list of unigram/bigrams and not a generator object
#df["title_clean"][1]

['fairness', 'academic', 'course_timetabling']

In [31]:
#Rewriting df variable with our cleaned dataset. 
df= pd.read_csv(dataset)

In [33]:
#df.head()

### Exploratory Data Visualization

In [283]:
### Word Cloud Generation Just to see:

# Import the wordcloud library
from wordcloud import WordCloud

# Join the different processed titles together.
#all_title = ','.join(list(df["title_clean"].values))

# Create a Title WordCloud object
cloud=WordCloud(background_color="black", max_words=1000)
                       
# Generate a word cloud
cloud.generate(df["title_clean"].to_string())

# Visualize the word cloud
cloud.to_image()


In [284]:
# Create an Abstract WordCloud object
cloud=WordCloud(background_color="black", max_words=1000)
                       
# Generate a word cloud
cloud.generate(df["abstract_clean"].to_string())

# Visualize the word cloud
cloud.to_image()


There are a couple of intersting things we can begin to think about and question with this simple visualization. 

- We see words we expect to see like "fairness" and "fair"
    - interstingly "fair" and "fairness" weren't lemmed to "fair" but we might consider them representing different uses/understandings 
        - where and how are we using "fairness" as opposed to "fair"?
    - Also "unfairness" is interestingly small when compared to fair and fairness. SO those seem to be used at the same amounts (possibly even in similar functions/interchangeably?) but "unfairness"is beng used differently somewhat. 
    
- The largest used terms are apparently functional terms liek "model" "system" "classification" etc. 
- It's interesting that words like "social" and "representation" have low occurence. 


    

## Topic Modelling - Latent Dirlecht Allocation (LDA)

Topic Modeling: A statistical modeling of the abstract topics that occur within a document (i.e. topics are "latent" within the document). In topic modeling we can understand a document as not only a collection of text but a distribution of "topic". 

Latent Dirlecht Allocation (LDA) is one type of model used in Topic Modelling. Our LDA model will build "topics" identify as cluster of words/tokens and then look at the distribution of topics throughout our documents. 

Our tasks in this section will be:
- Preprocessing 
    - We'll preparing our data by creating a **Bag of Words** representation of our cleaned text data
        
            - the bag of words will require us to:
                1. Make dictionary with all of the words in our corpus, numbered (and perhaps in alphabetical order)
                2. A count of how often each word appears in each document
                
- Model Fitting:
    - We'll build a pipeline 
Here we'll build a pipeline for fitting the LDA model. 

Returning to dictionary buildig let's look at our abstract data. Our LDA will require 3 things:

1. a dictionary - this will hold all the possible words (and in this second trial bi-grams) in our text with an id. 

2. a corpus - this is our colecltion of text, in this case the abstract_clean column

3. a Count of our term-document frequency. So the count of how many times a term (word or bigram) appears in a document (row of data)


In [234]:
#Making a Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer

cv= CountVectorizer(stop_words="english")

title_cv = cv.fit_transform(df['title_clean'])

dtm = pd.DataFrame(title_cv.toarray(), columns=cv.get_feature_names())
dtm.index = df.index

#Here is our document-term matrix 
dtm.head()

,academic,accountability,accuracy,achieve,active,acyclic,adaptation,adaptive,add,additive,address,adequate,advantage,adversarial,adversaries,advisor,aequitas,agenda,agent,agents,aggregation,agnostic,ai,algorithm,algorithmic,algorithmic_decision,algorithms,alignment,allocate,allocation,allocation_indivisible,allocations,analyse,analysis,applicability,application,applications,apply,approach,architecture_search,arm,aspect,assess,assignment,assumptions,astraea,attitudes,attribute,audit,auditors,automate,average,avoid,aware,aware_machine,balance,bandit,bandits,bank,base,bay,bayesian,behavior,behaviors,benchmarks,best,bias,binary,bio,boost,bound,bridge,budget,cake,capuchin,cardinality,case,causal,causal_effect,causal_model,causality,center,centric,certify,challenge,cheat,chest,chexclusion,churn,classification,classifier,classifiers,claudette,clauses,cluster,collaborative,collider,commerce,common,comparative,compare,comparison,comparisons,compas,compatibility,complexity,compositional,comprehension,computational,compute,computer,concentration,conditional,consequentialism,constrain,constraint,constraints,construct,construction,consumer,context,contextual,continuous,controllable,convex,convexity,core,counter,counterfactual,counterfactually,course_timetabling,cox,criteria,criterion,critically,critique,crowd,crowdsourcing,cryptocredit,curriculum,cut,dart,data,database,datasets,deal,decision,decision_make,decisions,decomposition,deep,deep_learn,deepfair,definitions,demand,dependent,design,detect,detection,detector,determine,dialogue,differences,differentiable,differential,differentially,direct,disability,discount,discover,discrimination,disentanglement,disparate,distribution,diversity,domain,domains,dominant,drive,dual,dynamic,dynamical,edge,education,educational,effect,efficiency,efficiency_sequenceability,efficient,effort,eligibility,eliminate,embeddings,embrace,empirical,employees,endogenous,enhance,ensemble,ensure,equality,equitable,equity,estimators,ethical,eu,evaluate,evaluation,everyday,evidence,examine,exchange,expectation,experiment,expert,explainability,explainable,explanation,explanations,expose,extend,extensible,eye,eyni,facial,fae,faht,fair,fair_correlation,fair_division,fair_representations,fairgroup,fairjudge,fairly,fairmixrep,fairmod,fairnas,fairness,fairrec,fairst,fairxgboost,fake,false,fare,farm,fat,feature,federate,field,filter,flexibly,fog,food,forensics,formalize,formally,formulations,framework,framework_measure,freeness,game,gap,gender,general_framework,generalization,generalize,gig,goals,goods,gradient,grammar,graph,graphical,greedy,group,groupwise,guidelines,handle,healthcare,heterogeneous,heuristics,hierarchically,high,hours,human,hyperparameter,ideal,identification,ignorance,imbalance,impact,impact_data,implement,impossibility,improve,improvements,incentive,individual,individually_fair,indivisible,indivisible_goods,industry,inference,influence,information,infra,inherent,inspire,instead,integrate,interpretability,intersectional,intersectionality,interventions,intuitive,items,jealousy,joint,judgement,justice,justicia,kidney,knowledge,lagrangian,language,latent,law,lead,learn,lecture,legal,lend,lens,lessons,level,leverage,lexicographic,lift,linear,localize,long,loop,lotteries,ltlf,machine,machine_learn,make,manage,manipulation,marginality,market,marketplaces,mathematical,matter,max,maximin,maximization,mean,measure,measurement,mechanism,medicaid,memory,meta,methodological,methods,metrics,min,minimization,miss,mitigate,mix_manna,ml,mobility,model,monotonic,multi,multi_agent,multi_arm,multiobjective,multiple,multistakeholder,multiwinner,naive,nash,need,negative,network,neural,neural_network,new,noise,noisy,non,nondeterministic,note,notion,notions,novel,object,objective,objectives,online,ontologies,openpredict,opportunistic,optimality,optimization,ordinal,organizations,outcome,outlier,pairwise,parity,path,pattern,pc,people_disabilities,perception,perform,personalization,personalize,perspective,physicists,pipelines,plan,

In [172]:
#Apparently Gensim requires a term document matrix, where the index is our terms and the columns are documents.
tdm=dtm_df.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217
academic,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
accountability,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
accuracy,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
achieve,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
active,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [196]:
df["title_clean"][1]

"['fairness', 'academic', 'course_timetabling']"

#### Working with Title Data:

In [254]:
from gensim.utils import simple_preprocess


def preprocess(text):
   result = []
   for token in gensim.utils.simple_preprocess(text):
      #if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
       if len(token) > 1: 
         result.append(token)
   return result

processed = df["title_clean"].map(preprocess)


dictionary = gensim.corpora.Dictionary(processed)


In [243]:
from gensim import matutils, models
import scipy.sparse

#transposing our document term matrix to a form gensim accepts
#so wher before x axis was a document and y axis was a word, these have been switched
tdm=dtm.transpose()

sparse_counts = scipy.sparse.csr_matrix(dtm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [259]:
#texts = [[word for word in df["title_clean"].split()]] 
#for document in documents]
#dictionary = corpora.Dictionary(texts)
#corpus = [dictionary.doc2bow(text) for text in texts]



#title_dict = gensim.corpora.Dictionary(df["title_clean"])

#creating a bag of words of just title information
#title_corpus = [title_dict.doc2bow(x) for x in df["title_clean"]]
#title_corpus = [title_dict.doc2bow(x) for x in texts]

#Instantiating LDA model
lda_t = gensim.models.ldamodel.LdaModel(id2word=dictionary, corpus=corpus, num_topics=5, passes=10)

#Putting our Title topics in a dataframe
title_topics = pd.DataFrame(lda_t.print_topics(num_words=10))
title_topics.columns= ["topics", "terms"]
title_topics

,topics,terms
0,0,"0.026*""perspective"" + 0.026*""learn"" + 0.026*""reframing"" + 0.022*""explainable"" + 0.021*""lecture"" + 0.020*""food"" + 0.020*""outcome"" + 0.020*""model"" + 0.020*""years"" + 0.020*""safeguard"""
1,1,"0.026*""service"" + 0.020*""fairly"" + 0.018*""criteria"" + 0.017*""optimality"" + 0.017*""bound"" + 0.017*""lend"" + 0.017*""play"" + 0.016*""cake"" + 0.016*""robots"" + 0.016*""supervise_learn"""
2,2,"0.027*""cut"" + 0.025*""veil"" + 0.024*""social"" + 0.024*""public"" + 0.023*""ordinal"" + 0.023*""kidney"" + 0.021*""intersectional"" + 0.020*""handle"" + 0.020*""vote"" + 0.020*""toolkit"""
3,3,"0.029*""query"" + 0.021*""groupwise"" + 0.021*""measurement"" + 0.021*""term"" + 0.020*""product"" + 0.019*""decision_make"" + 0.018*""fare"" + 0.018*""formalize"" + 0.018*""alignment"" + 0.018*""aggregation"""
4,4,"0.029*""items"" + 0.021*""software"" + 0.021*""task"" + 0.020*""potentially"" + 0.019*""transportation"" + 0.018*""diversity"" + 0.016*""parity"" + 0.016*""unfair"" + 0.016*""detector"" + 0.016*""robust"""


In [185]:
df["title_clean"][1]

"['fairness', 'academic', 'course_timetabling']"

#### Working with Abstract Data

In [263]:
#df.head()

In [260]:
#Now doing everything we just did but for abstract data:

#First getting a document term matrix and setting a sparse matrix corpus
#cv = CountVectorizer(stop_words="english")

ab_cv = cv.fit_transform(df['abstract_clean'])

ab_dtm = pd.DataFrame(ab_cv.toarray(), columns=cv.get_feature_names())
ab_dtm.index = df.index

In [261]:
#Next creating a sparse matrix corpus
s_counts = scipy.sparse.csr_matrix(ab_dtm)
ab_corpus = matutils.Sparse2Corpus(s_counts)

In [277]:
#Creating our dictionary of tokens with ids
def preprocess(text):
   result = []
   for token in gensim.utils.simple_preprocess(text):
      
       if len(token) > 1: 
         result.append(token)
   return result

abstracts = df["abstract_clean"].map(preprocess)

ab_dictionary = gensim.corpora.Dictionary(processed)


In [288]:
#Finally instantiating LDA model and fitting to our abstract data. 
lda = gensim.models.ldamodel.LdaModel(id2word=ab_dictionary, corpus=ab_corpus, num_topics=3, passes=30)

#Putting our Title topics in a dataframe
abstract_topics = pd.DataFrame(lda.print_topics(num_words=10))
abstract_topics.columns= ["topics", "terms"]
abstract_topics

,topics,terms
0,0,"0.031*""deep"" + 0.031*""commerce"" + 0.024*""attitudes"" + 0.021*""cut"" + 0.021*""non"" + 0.021*""welfare"" + 0.020*""public"" + 0.020*""nash"" + 0.018*""claudette"" + 0.017*""exchange"" + 0.017*""convexity"" + 0.017*""veil"" + 0.017*""tolerant"" + 0.017*""metrics"" + 0.016*""safeguard"" + 0.016*""mechanism"" + 0.016*""recommendation"" + 0.015*""apply"" + 0.015*""generalize"" + 0.015*""maximin"""
1,1,"0.028*""embrace"" + 0.027*""intersectional"" + 0.024*""critically"" + 0.024*""multiple"" + 0.023*""model"" + 0.022*""application"" + 0.022*""allocate"" + 0.019*""causal_model"" + 0.019*""toolkit"" + 0.018*""analysis"" + 0.018*""detector"" + 0.018*""potentially"" + 0.017*""regression"" + 0.016*""handle"" + 0.016*""classification"" + 0.016*""constraints"" + 0.015*""applications"" + 0.014*""collaborative"" + 0.014*""prediction"" + 0.014*""items"""
2,2,"0.030*""judgement"" + 0.026*""responsible"" + 0.025*""service"" + 0.024*""un"" + 0.021*""predictive"" + 0.020*""goals"" + 0.019*""group"" + 0.019*""counterfactual"" + 0.018*""query"" + 0.018*""fair_division"" + 0.018*""educational"" + 0.017*""challenge"" + 0.017*""problem"" + 0.017*""binary"" + 0.017*""price"" + 0.017*""decision_make"" + 0.017*""value"" + 0.016*""recall"" + 0.016*""dominant"" + 0.016*""capuchin"""


In [34]:
#Saving everything
#abstract_topics.to_csv("/Users/aster/Desktop/fall_2020/ML/final_project/pratt_ml_final_project/data/abstract_topics.csv")

In [291]:
df.shape

(218, 18)